In [37]:
import numpy as np
import json
import random
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

import sys

sys.path.append('../src')

from config.config import PATHS

In [38]:
# No. of agents
N = 12
# No. of available spots
s = 4

In [39]:
import matplotlib.colors as mcolors
named_colors = mcolors.CSS4_COLORS
color_names = list(named_colors.keys())
random.seed(1234)
random.shuffle(color_names)
shape_names = ['dot','diamond','square','triangle','star','triangleDown']

def bin_to_decimal(bin_str):
    return int(bin_str, 2)

def get_color(bin_str):
    decimal = bin_to_decimal(bin_str)
    return color_names[decimal % len(color_names)]

def get_shape(bin_str):
    decimal = bin_to_decimal(bin_str)
    return shape_names[decimal % len(shape_names)]

In [40]:
file_name = PATHS['graphs'] / f'graph_data_N{N:d}s{s:d}.json'
with open(file_name, 'r') as f:
  struct = json.load(f)
#print(json.dumps(struct, indent = 2))

In [41]:
Npat = 0
edges = []
e_colors = {}
n_colors = {}
n_shapes = {}
for n1 in struct[Npat]:
    n_colors[n1] = get_color(''.join(i for i in struct[Npat][n1]["pattern"]))
    n_shapes[n1] = get_shape(''.join(i for i in struct[Npat][n1]["pattern"]))
    for n2 in struct[Npat][str(n1)]["neigh"]:  
        if len(struct[Npat][str(n1)]["strat"]) > 1:
            edges.append((n2, n1))

            copy_strat = False
            if len(struct[Npat][str(n1)]["strat"]) == 2:
                copy_strat = (struct[Npat][str(n1)]["strat"]['0'] == '0') and (struct[Npat][str(n1)]["strat"]['1'] == '1')
                if copy_strat:
                    e_colors[f'{n2}-{n1}'] = 'darkturquoise'
                else:
                    e_colors[f'{n2}-{n1}'] = 'black'
            else:
                e_colors[f'{n2}-{n1}'] = 'black'

In [42]:
# print(edges)
# print(e_colors)
# print(n_colors)

In [43]:
n_colors['3']

'coral'

In [44]:
DG = nx.DiGraph()
DG.add_nodes_from([str(i) for i in range(N)])
DG.add_edges_from(edges)
for node in DG.nodes:
    DG.nodes[node]['color'] = n_colors[node]

In [45]:
list(DG.nodes(data=True))

[('0', {'color': 'coral'}),
 ('1', {'color': 'goldenrod'}),
 ('2', {'color': 'goldenrod'}),
 ('3', {'color': 'coral'}),
 ('4', {'color': 'khaki'}),
 ('5', {'color': 'khaki'}),
 ('6', {'color': 'khaki'}),
 ('7', {'color': 'khaki'}),
 ('8', {'color': 'coral'}),
 ('9', {'color': 'goldenrod'}),
 ('10', {'color': 'goldenrod'}),
 ('11', {'color': 'coral'})]

In [46]:
dnet = Network("400px","400px",notebook=True,directed=True)
dnet.from_nx(DG)
for edge in dnet.edges:
    edge['color'] = e_colors[f"{edge['from']}-{edge['to']}"]
for node in dnet.nodes:
    node['color'] = {'background': n_colors[node['id']], 'border': 'black'}
    node['shape'] = n_shapes[node['id']]

In [47]:
out_file = PATHS['html'] / f'graph_N{N:d}s{s:d}.html'
dnet.toggle_physics(True)
dnet.show(str(out_file))

/home/carlos/Documents/Information/edgar_german/repo/alternation_EFP/data/html/graph_N12s4.html


### Cycles

In [48]:
# Detect simple cycles
cycles = list(nx.simple_cycles(DG))
print(f"Number of simple cycles: {len(cycles)}")
for cycle in cycles:
    print(" -> ".join(cycle))

Number of simple cycles: 2
4 -> 8 -> 10
3 -> 9 -> 6


In [49]:
# Number of 1s in the cycles
for c in range(len(cycles)):
    cycle = cycles[c]
    ones_count = 0
    for i in cycle:
        ones_count += int(struct[Npat][i]["pattern"][0])
    print(f"Cycle {c}: {ones_count} ones, Pattern: {' -> '.join(cycle)}")

Cycle 0: 1 ones, Pattern: 4 -> 8 -> 10
Cycle 1: 1 ones, Pattern: 3 -> 9 -> 6


In [50]:
len(struct)

10

In [ ]:
for Np in range(len(struct)):
    for i in range(N):
        struct[Np][str(i)]['cycle'] = -1
        struct[Np][str(i)]['ones in cycle'] = 0
        for c in range(len(cycles)):
            if str(i) in cycles[c]:
                struct[Np][str(i)]['cycle'] = c
                struct[Np][str(i)]['ones in cycle'] = sum(int(struct[Np][j]["pattern"][0]) for j in cycles[c])
                break
        #print(struct[Np][str(i)])


In [52]:
graph_path = PATHS['graphs'] / f"graph_data_N{N:d}s{s:d}.json"
with open(graph_path, 'w') as json_file:
    json.dump(struct, json_file)